# Regression Model

In [1]:
import pandas as pd
import numpy as np
import missingno as msno

import warnings
warnings.filterwarnings('ignore')

## Import data

In [2]:
df_click_purchase = pd.read_csv('data/clickstreams_with_purchase.csv')
df_click_purchase.head()

,customer_id,date,time,page_viewed,product_id,ordered
0,1436364061643577713,2018-07-16,14:37:44,HOMEPAGE,0,0.0
1,1436364061643577713,2018-07-16,14:40:44,PRODUCTSIMPLESKU,205401096165894320,0.0
2,1436364061643577713,2018-07-16,14:42:39,PRODUCTSIMPLESKU,9220344733964903803,0.0
3,1436364061643577713,2018-07-16,14:42:53,PRODUCTSIMPLESKU,9220344733964903803,0.0
4,1436364061643577713,2018-07-16,14:44:46,KEYWORDSEARCH,0,0.0


In [4]:
df_orders = pd.read_csv('data/orders.csv')
print(df_orders.shape)
df_orders.head()

(137537, 9)


,order_product_id,order_id,wayfair_product_id,wayfair_website_id,order_date,order_time,customer_id,customer_zip,quantity_ordered
0,8.344710e+18,1.073040e+17,8.089580e+18,Wayfair,7/19/18,9:48:02,3.243950e+18,29464,1
1,7.897050e+18,9.115090e+17,4.407090e+18,Wayfair,7/21/18,15:04:23,7.606760e+18,10928,1
2,3.742700e+17,9.115090e+17,1.327620e+18,Wayfair,7/21/18,15:04:23,7.606760e+18,10928,1
3,7.072690e+18,5.760940e+18,7.293640e+18,Wayfair,7/22/18,21:56:42,3.106940e+18,6712,1
4,7.307240e+18,2.708890e+18,7.264500e+18,Wayfair,7/21/18,12:55:20,1.390570e+18,2468,1


In [5]:
df_orders1 = df_orders.rename(columns={'wayfair_product_id':'product_id'})
df_orders1.head()

,order_product_id,order_id,product_id,wayfair_website_id,order_date,order_time,customer_id,customer_zip,quantity_ordered
0,8.344710e+18,1.073040e+17,8.089580e+18,Wayfair,7/19/18,9:48:02,3.243950e+18,29464,1
1,7.897050e+18,9.115090e+17,4.407090e+18,Wayfair,7/21/18,15:04:23,7.606760e+18,10928,1
2,3.742700e+17,9.115090e+17,1.327620e+18,Wayfair,7/21/18,15:04:23,7.606760e+18,10928,1
3,7.072690e+18,5.760940e+18,7.293640e+18,Wayfair,7/22/18,21:56:42,3.106940e+18,6712,1
4,7.307240e+18,2.708890e+18,7.264500e+18,Wayfair,7/21/18,12:55:20,1.390570e+18,2468,1


In [6]:
df_products = pd.read_csv('data/products.csv')
print(df_products.shape)
df_products.head()

(498673, 13)


,website_name,product_id,product_name,category_name,description,manufacturer_name,weight,dimension_1,dimension_2,dimension_3,num_reviews,onsite_price,on_promotion
0,Joss US,2.790000e+18,Westgate Upholstered Panel Bed,Beds,The 1882 upholstered bed is sure to appeal to ...,Pulaski Furniture,116.0,59.25,87.50,68.75,7.0,409.99,0
1,Joss US,8.190000e+18,3 Piece Leah Bowl Set,"Boxes, Bins, Baskets, & Buckets",Stack green apples on the breakfast bar or dis...,Three Hands Co.,30.8,23.00,28.35,14.17,0.0,89.99,1
2,Joss US,7.240000e+18,Butter Cups/Ranunculus Sunshine Bulb,Planting & Seeding,Start your own backyard garden or add spring s...,Van Zyverden Inc,1.0,16.00,3.00,4.00,0.0,16.99,0
3,Joss US,4.110000e+18,Freesias Double Blooming Bulb,Planting & Seeding,Start your own backyard garden or add spring s...,Van Zyverden Inc,1.0,24.00,2.00,2.00,2.0,9.99,0
4,Joss US,7.670000e+18,Grand Freesia/Ranunculus Blend Bulb,Planting & Seeding,Start your own backyard garden or add spring s...,Van Zyverden Inc,1.0,36.00,3.00,2.00,2.0,9.99,0


In [7]:
df_taxes = pd.read_csv('data/taxes.csv')
df_taxes.head()

,state_fips,state,zipcode,agi_stub,num_returns,num_single,num_joint,num_hoh,num_dependents,num_elderly,...,N19300,A19300,N19500,A19500,N04800,A04800,N07220,A07220,N11901,A11901
0,1,AL,0,1,815440,477700,105350,221200,491310,150920,...,25020,5950,46930,74541,17910,82471,27490,15002,0,0
1,1,AL,0,2,495830,211930,142340,128890,360480,109680,...,64360,17611,102740,243301,44430,292180,43580,45248,0,0
2,1,AL,0,3,263390,83420,137870,36340,182880,75570,...,68320,19479,99920,337719,37110,278272,22780,29166,0,0
3,1,AL,0,4,167190,29420,124060,10610,130160,51540,...,58540,18167,79980,362811,24780,202113,17250,22630,0,0
4,1,AL,0,5,217440,20240,188080,4880,195990,68080,...,117980,44270,151920,1065202,34030,298272,20970,29469,390,155


## Extract features from df_product

In [26]:
columns = ['product_id','weight','num_reviews','onsite_price','on_promotion','description','category_name','manufacturer_name']
df_products1 = df_products.loc[:,columns]
df_products1.head()

,product_id,weight,num_reviews,onsite_price,on_promotion,description,category_name,manufacturer_name
0,2.790000e+18,116.0,7.0,409.99,0,The 1882 upholstered bed is sure to appeal to ...,Beds,Pulaski Furniture
1,8.190000e+18,30.8,0.0,89.99,1,Stack green apples on the breakfast bar or dis...,"Boxes, Bins, Baskets, & Buckets",Three Hands Co.
2,7.240000e+18,1.0,0.0,16.99,0,Start your own backyard garden or add spring s...,Planting & Seeding,Van Zyverden Inc
3,4.110000e+18,1.0,2.0,9.99,0,Start your own backyard garden or add spring s...,Planting & Seeding,Van Zyverden Inc
4,7.670000e+18,1.0,2.0,9.99,0,Start your own backyard garden or add spring s...,Planting & Seeding,Van Zyverden Inc


In [37]:
df = pd.merge(df_orders1, df_products1, on='product_id')
df = df.drop(columns=['order_product_id','order_id','wayfair_website_id'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80123 entries, 0 to 80122
Data columns (total 13 columns):
product_id           80123 non-null float64
order_date           80123 non-null object
order_time           80123 non-null object
customer_id          80123 non-null float64
customer_zip         80123 non-null object
quantity_ordered     80123 non-null int64
weight               80123 non-null float64
num_reviews          80123 non-null float64
onsite_price         80123 non-null float64
on_promotion         80123 non-null int64
description          80123 non-null object
category_name        80123 non-null object
manufacturer_name    80123 non-null object
dtypes: float64(5), int64(2), object(6)
memory usage: 8.6+ MB


In [38]:
df['order_value'] = df.quantity_ordered*df.onsite_price
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80123 entries, 0 to 80122
Data columns (total 14 columns):
product_id           80123 non-null float64
order_date           80123 non-null object
order_time           80123 non-null object
customer_id          80123 non-null float64
customer_zip         80123 non-null object
quantity_ordered     80123 non-null int64
weight               80123 non-null float64
num_reviews          80123 non-null float64
onsite_price         80123 non-null float64
on_promotion         80123 non-null int64
description          80123 non-null object
category_name        80123 non-null object
manufacturer_name    80123 non-null object
order_value          80123 non-null float64
dtypes: float64(6), int64(2), object(6)
memory usage: 9.2+ MB


## Features int df_taxes

In [39]:
df.customer_zip=df.customer_zip.astype(int)
df = df.rename(columns={'customer_zip':'zipcode'})



In [40]:
df_taxes1 = df_taxes.loc[:,['state','zipcode','num_returns']]
df_taxes1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179796 entries, 0 to 179795
Data columns (total 3 columns):
state          179796 non-null object
zipcode        179796 non-null int64
num_returns    179796 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.1+ MB


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80123 entries, 0 to 80122
Data columns (total 14 columns):
product_id           80123 non-null float64
order_date           80123 non-null object
order_time           80123 non-null object
customer_id          80123 non-null float64
zipcode              80123 non-null int64
quantity_ordered     80123 non-null int64
weight               80123 non-null float64
num_reviews          80123 non-null float64
onsite_price         80123 non-null float64
on_promotion         80123 non-null int64
description          80123 non-null object
category_name        80123 non-null object
manufacturer_name    80123 non-null object
order_value          80123 non-null float64
dtypes: float64(6), int64(3), object(5)
memory usage: 9.2+ MB


In [42]:
df_taxes1=df_taxes1.groupby('zipcode').mean()
df = pd.merge(df,df_taxes1,on='zipcode')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79080 entries, 0 to 79079
Data columns (total 15 columns):
product_id           79080 non-null float64
order_date           79080 non-null object
order_time           79080 non-null object
customer_id          79080 non-null float64
zipcode              79080 non-null int64
quantity_ordered     79080 non-null int64
weight               79080 non-null float64
num_reviews          79080 non-null float64
onsite_price         79080 non-null float64
on_promotion         79080 non-null int64
description          79080 non-null object
category_name        79080 non-null object
manufacturer_name    79080 non-null object
order_value          79080 non-null float64
num_returns          79080 non-null float64
dtypes: float64(7), int64(3), object(5)
memory usage: 9.7+ MB


## Dummies for categorical parameters

In [43]:
df.order_date=df.order_date.str.replace('\d\/16\/\d+','Monday')
df.order_date=df.order_date.str.replace('\d\/17\/\d+','Tuesday')
df.order_date=df.order_date.str.replace('\d\/18\/\d+','Wednesday')
df.order_date=df.order_date.str.replace('\d\/19\/\d+','Thursday')
df.order_date=df.order_date.str.replace('\d\/20\/\d+','Friday')
df.order_date=df.order_date.str.replace('\d\/21\/\d+','Saturday')
df.order_date=df.order_date.str.replace('\d\/22\/\d+','Sunday')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79080 entries, 0 to 79079
Data columns (total 15 columns):
product_id           79080 non-null float64
order_date           79080 non-null object
order_time           79080 non-null object
customer_id          79080 non-null float64
zipcode              79080 non-null int64
quantity_ordered     79080 non-null int64
weight               79080 non-null float64
num_reviews          79080 non-null float64
onsite_price         79080 non-null float64
on_promotion         79080 non-null int64
description          79080 non-null object
category_name        79080 non-null object
manufacturer_name    79080 non-null object
order_value          79080 non-null float64
num_returns          79080 non-null float64
dtypes: float64(7), int64(3), object(5)
memory usage: 9.7+ MB


In [44]:
df2=pd.concat([df,pd.get_dummies(df.order_date)],axis=1)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79080 entries, 0 to 79079
Data columns (total 22 columns):
product_id           79080 non-null float64
order_date           79080 non-null object
order_time           79080 non-null object
customer_id          79080 non-null float64
zipcode              79080 non-null int64
quantity_ordered     79080 non-null int64
weight               79080 non-null float64
num_reviews          79080 non-null float64
onsite_price         79080 non-null float64
on_promotion         79080 non-null int64
description          79080 non-null object
category_name        79080 non-null object
manufacturer_name    79080 non-null object
order_value          79080 non-null float64
num_returns          79080 non-null float64
Friday               79080 non-null uint8
Monday               79080 non-null uint8
Saturday             79080 non-null uint8
Sunday               79080 non-null uint8
Thursday             79080 non-null uint8
Tuesday              79080 non-n

In [45]:
for i in range(0,12):
    df2.order_time=df2.order_time.str.replace('^'+str(i)+':\S+','morning')
for i in range(12,18):
    df2.order_time=df2.order_time.str.replace('^'+str(i)+':\S+','afternoon')
for i in range(18,24):  
    df2.order_time=df2.order_time.str.replace('^'+str(i)+':\S+','evening')
df2.order_time.value_counts()    

afternoon    28379
evening      27860
morning      22841
Name: order_time, dtype: int64

In [46]:
df2=pd.concat([df2,pd.get_dummies(df2.order_time)],axis=1)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79080 entries, 0 to 79079
Data columns (total 25 columns):
product_id           79080 non-null float64
order_date           79080 non-null object
order_time           79080 non-null object
customer_id          79080 non-null float64
zipcode              79080 non-null int64
quantity_ordered     79080 non-null int64
weight               79080 non-null float64
num_reviews          79080 non-null float64
onsite_price         79080 non-null float64
on_promotion         79080 non-null int64
description          79080 non-null object
category_name        79080 non-null object
manufacturer_name    79080 non-null object
order_value          79080 non-null float64
num_returns          79080 non-null float64
Friday               79080 non-null uint8
Monday               79080 non-null uint8
Saturday             79080 non-null uint8
Sunday               79080 non-null uint8
Thursday             79080 non-null uint8
Tuesday              79080 non-n

In [47]:
df2['description_length'] = df2.description.str.len()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79080 entries, 0 to 79079
Data columns (total 26 columns):
product_id            79080 non-null float64
order_date            79080 non-null object
order_time            79080 non-null object
customer_id           79080 non-null float64
zipcode               79080 non-null int64
quantity_ordered      79080 non-null int64
weight                79080 non-null float64
num_reviews           79080 non-null float64
onsite_price          79080 non-null float64
on_promotion          79080 non-null int64
description           79080 non-null object
category_name         79080 non-null object
manufacturer_name     79080 non-null object
order_value           79080 non-null float64
num_returns           79080 non-null float64
Friday                79080 non-null uint8
Monday                79080 non-null uint8
Saturday              79080 non-null uint8
Sunday                79080 non-null uint8
Thursday              79080 non-null uint8
Tuesday     

In [55]:
category_count = df2.groupby('category_name')['description_length'].count().reset_index(name = 'category_count')
manu_count = df2.groupby('manufacturer_name')['description_length'].count().reset_index(name = 'manu_count')


In [57]:
df3 = pd.merge(df2,category_count,on='category_name')
df3 = pd.merge(df3,manu_count,on='manufacturer_name')
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79080 entries, 0 to 79079
Data columns (total 28 columns):
product_id            79080 non-null float64
order_date            79080 non-null object
order_time            79080 non-null object
customer_id           79080 non-null float64
zipcode               79080 non-null int64
quantity_ordered      79080 non-null int64
weight                79080 non-null float64
num_reviews           79080 non-null float64
onsite_price          79080 non-null float64
on_promotion          79080 non-null int64
description           79080 non-null object
category_name         79080 non-null object
manufacturer_name     79080 non-null object
order_value           79080 non-null float64
num_returns           79080 non-null float64
Friday                79080 non-null uint8
Monday                79080 non-null uint8
Saturday              79080 non-null uint8
Sunday                79080 non-null uint8
Thursday              79080 non-null uint8
Tuesday     

In [59]:
df3['category_rate'] = df3.category_count/(len(df3))
df3['manufacturer_rate'] = df3.manu_count/(len(df3))
df3.head()

,product_id,order_date,order_time,customer_id,zipcode,quantity_ordered,weight,num_reviews,onsite_price,on_promotion,...,Tuesday,Wednesday,afternoon,evening,morning,description_length,category_count,manu_count,category_rate,manufacturer_rate
0,7.842510e+15,Thursday,afternoon,6.231540e+18,61832,1,62.0,511.0,157.31,1,...,0,0,1,0,0,300,154,624,0.001947,0.007891
1,7.842510e+15,Wednesday,afternoon,3.839240e+18,8858,1,62.0,511.0,157.31,1,...,0,1,1,0,0,300,154,624,0.001947,0.007891
2,7.842510e+15,Saturday,morning,4.570650e+18,28054,1,62.0,511.0,157.31,1,...,0,0,0,0,1,300,154,624,0.001947,0.007891
3,7.842510e+15,Wednesday,evening,2.495660e+18,78233,1,62.0,511.0,157.31,1,...,0,1,0,1,0,300,154,624,0.001947,0.007891
4,7.842510e+15,Sunday,morning,7.211280e+18,7036,1,62.0,511.0,157.31,1,...,0,0,0,0,1,300,154,624,0.001947,0.007891


In [62]:
data = df3.drop(columns=['order_date','order_time','description','description','category_name','manufacturer_name',
                        'category_count','manu_count'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79080 entries, 0 to 79079
Data columns (total 23 columns):
product_id            79080 non-null float64
customer_id           79080 non-null float64
zipcode               79080 non-null int64
quantity_ordered      79080 non-null int64
weight                79080 non-null float64
num_reviews           79080 non-null float64
onsite_price          79080 non-null float64
on_promotion          79080 non-null int64
order_value           79080 non-null float64
num_returns           79080 non-null float64
Friday                79080 non-null uint8
Monday                79080 non-null uint8
Saturday              79080 non-null uint8
Sunday                79080 non-null uint8
Thursday              79080 non-null uint8
Tuesday               79080 non-null uint8
Wednesday             79080 non-null uint8
afternoon             79080 non-null uint8
evening               79080 non-null uint8
morning               79080 non-null uint8
description_lengt

In [63]:
data.to_csv('regression_data.csv',index=False)

linear regression, random forest, neural 

# AutoML regression

## TPOT

In [64]:
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split

/anaconda3/lib/python3.6/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap_external.py:426: ImportWarning: Not importing directory /anaconda3/lib/python3.6/site-packages/mpl_toolkits: missing __init__
  _warnings.warn(msg.format(portions[0]

In [65]:
df = pd.read_csv('regression_data.csv',)
df.head()

,product_id,customer_id,zipcode,quantity_ordered,weight,num_reviews,onsite_price,on_promotion,order_value,num_returns,...,Sunday,Thursday,Tuesday,Wednesday,afternoon,evening,morning,description_length,category_rate,manufacturer_rate
0,7.842510e+15,6.231540e+18,61832,1,62.0,511.0,157.31,1,157.31,2350.000000,...,0,1,0,0,1,0,0,300,0.001947,0.007891
1,7.842510e+15,3.839240e+18,8858,1,62.0,511.0,157.31,1,157.31,63.333333,...,0,0,0,1,1,0,0,300,0.001947,0.007891
2,7.842510e+15,4.570650e+18,28054,1,62.0,511.0,157.31,1,157.31,2795.000000,...,0,0,0,0,0,0,1,300,0.001947,0.007891
3,7.842510e+15,2.495660e+18,78233,1,62.0,511.0,157.31,1,157.31,3675.000000,...,0,0,0,1,0,1,0,300,0.001947,0.007891
4,7.842510e+15,7.211280e+18,7036,1,62.0,511.0,157.31,1,157.31,3731.666667,...,1,0,0,0,0,0,1,300,0.001947,0.007891


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79080 entries, 0 to 79079
Data columns (total 23 columns):
product_id            79080 non-null float64
customer_id           79080 non-null float64
zipcode               79080 non-null int64
quantity_ordered      79080 non-null int64
weight                79080 non-null float64
num_reviews           79080 non-null float64
onsite_price          79080 non-null float64
on_promotion          79080 non-null int64
order_value           79080 non-null float64
num_returns           79080 non-null float64
Friday                79080 non-null int64
Monday                79080 non-null int64
Saturday              79080 non-null int64
Sunday                79080 non-null int64
Thursday              79080 non-null int64
Tuesday               79080 non-null int64
Wednesday             79080 non-null int64
afternoon             79080 non-null int64
evening               79080 non-null int64
morning               79080 non-null int64
description_lengt

In [131]:
features = df.drop(columns=['product_id','customer_id','order_value']).values
target = df['order_value'].values

In [136]:
X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                    train_size=0.75, test_size=0.25)
tpot = TPOTRegressor(generations=5, population_size=20, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

Generation 1 - Current best internal CV score: -25962.50794232748
Generation 2 - Current best internal CV score: -12765.153915385596
Generation 3 - Current best internal CV score: -12765.153915385596
Generation 4 - Current best internal CV score: -10906.2854825143
Generation 5 - Current best internal CV score: -10621.154810500748

Best pipeline: DecisionTreeRegressor(input_matrix, max_depth=10, min_samples_leaf=2, min_samples_split=13)
-6087.0654196421365
